# TP4
## Grupo 15

Carlos Eduardo Da Silva Machado A96936

Gonçalo Manuel Maia de Sousa A97485

## Exercício 4.1

### Descrição do Problema

No contexto do sistema de travagem ABS (“Anti-Lock Breaking System”), pretende-se construir um autómato híbrido que descreva o sistema e que  possa ser usado para verificar as suas propriedades dinâmicas.

    
1. A componente discreta do autómato contém os modos:  `Start`,  `Free`,  `Stopping`, `Blocked`, e `Stopped`. No modo `Free`  não existe qualquer força de travagem; no modo `Stopping` aplica-se a força de travagem alta; no modo `Blocked` as rodas estão bloqueadas em relação ao corpo mas o veículo  desloca-se; no modo `Stopped` o veículo está imobilizado.
2. A componente contínua  do autómato usa variáveis contínuas $\,V,v\,$ para descrever a  `velocidade do corpo`  do veículo em relação ao solo e a `velocidade linear das rodas` também em relação ao solo. 
        Assume-se que o sistema de travagem exerce uma força de atrito  nos travões proporcional à diferença das duas velocidades.  A dinâmica contínua está descrita  abaixo no bloco Equaçoes de Fluxo.
3. Os “switchs” (“jumps”) são a componente de projeto deste trabalho; cabe ao aluno definir quais devem ser estas  condições de modo a que o sistema tenha um comportamento desejável: imobilize-se depressa e não “derrape” muito.
4. É imprescindível evitar que o sistema tenha “trajetórias de Zenão”. Isto é, sequências  infinitas de transições  entre dois modos em intervalos de tempo  que tendem para zero mas nunca alcançam zero.

### Abordagem do Problema

1. Componentes Contínuas:
- velocidade linear das rodas
- velocidade do corpo (veículo)
- tempo
- timer

2. Componentes Discretas:
- Modo

3. Constantes:
- força de atrito do solo no modo block (baixa) e força de atrito nos restantes modos (alta)
- constante de proporcionalidade no modo stopping (alta) e nos restantes (baixa)
- velocidade inicial
- peso




eventos (aka nodos): Start, Free, Stopping, Blocked e Stopped
- Start -> Inicio
- Free -> A força de travagem não é exercida sobre o veículo
- Stopping -> É aplicada uma força de travagem alta sobre o veículo
- Blocked -> As rodas estão travadas, porém o veículo continua a mover-se
- Stopped -> O veículo pára

Jumps (ter em atenção o comportamento desejável: imobilize-se depressa e não “derrape” muito):
- Start -> Free (Inicia para um movimento livre)
- Stopping -> Blocked (O véiculo começa a derrapar)
- Blocked -> Free
- Stopping -> blocked



## Código Python

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import *

Start = Int(-1)
Free = Int(0)
Stopping = Int(1)
Blocked = Int(2)
Stopped = Int(3)

V0 = 10
aBlocked = 0.1
a = 1
cStopping = 10
c = 2
Tau = 1.0
P = 10
dif = 0.5 # intervalo entre velocidade do corpo e das rodas
delta = 0.01 # intervalo entre os tempos

In [2]:
def declare(i):
    s = {}
    # Discretas
    s['m'] = Symbol('m'+str(i),INT) # Modo
    # Contínuas
    s['t'] = Symbol('t'+str(i),REAL) # Tempo
    s['V'] = Symbol('V'+str(i),REAL) # Velocidade do corpo
    s['v'] = Symbol('v'+str(i),REAL) # velocidade linear das rodas
    s['timer'] = Symbol('timer'+str(i),REAL) # Timer
    return s

In [3]:
def init(s):
    return And(Equals(s['m'], Start),Equals(s['t'],Real(0)), Equals(s['V'], Real(V0)), Equals(s['v'], Real(V0)))

In [4]:
def trans(s,p):
    # untimed
    
    StartToFree = And(Equals(s['m'], Start),
                      Equals(p['m'], Free),
                      Equals(p['timer'],s['t'] + Real(Tau)),
                      Equals(p['V'], s['V']),
                      Equals(p['v'], s['v']),
                      Equals(p['t'], s['t']))
    
    FreeToStopping = And(Equals(s['m'], Free),
                         Equals(p['m'], Stopping),
                         Equals(p['V'], s['V']),
                         Equals(p['v'], s['v']),
                         Equals(p['t'], s['t']),
                         s['t'] >= p['timer'],
                         Equals(p['timer'],s['timer']))
    
    StoppingToBlocked = And(Equals(s['m'], Stopping),
                            Equals(p['m'], Blocked),
                            s['V']-s['v'] < Real(dif),
                            Equals(p['V'], s['V']),
                            Equals(p['v'], s['v']),
                            Equals(p['t'], s['t']),
                            s['t'] >= p['timer'],
                            Equals(p['timer'],s['t'] + Real(Tau)))
    
    BlockedToFree = And(Equals(s['m'], Blocked),
                        Equals(p['m'], Free),
                        Equals(p['V'], s['V']),
                        Equals(p['v'], s['v']),
                        Equals(p['timer'],s['t'] + Real(Tau)),
                        Equals(p['t'], s['t']))
    
    StoppingToStopped = And(Equals(s['m'], Stopping),
                            Equals(p['m'], Stopped),
                            Equals(p['V'], s['V']),
                            Equals(p['v'], s['v']),
                            Equals(p['t'], s['t']),
                            p['V'] < Real(dif),
                            p['v'] < Real(dif),
                            Equals(p['timer'],s['timer']))
    
    # timed
    
    FreeFree = And(Equals(s['m'], Free),
                   Equals(p['m'], Free),
                   Equals(p['V']-s['V'], -(Real(c) * (s['V'] - s['v'])) * (p['t'] - s['t'])),
                   Equals(p['v']-s['v'], (-(Real(a) * Real(P)) + (Real(c) * (s['V'] - s['v']))) * (p['t'] - s['t'])),
                   p['t'] < p['timer'],
                   Equals(p['timer'],s['timer']), 
                   p['t']>s['t']+delta)
    
    StoppingStopping = And(Equals(s['m'], Stopping),
                           Equals(p['m'], Stopping),
                           Equals(p['V'], -(Real(cStopping) * (s['V'] - s['v']))),
                           Equals(p['v'], -(Real(a) * Real(P)) + (Real(cStopping) * (s['V'] - s['v']))),
                           Equals(p['v'], s['t'] + (-Real(a) * Real(P)) * (p['t']-s['t'])),
                           Equals(p['timer'],s['timer']),
                           p['t']>s['t']+delta)
    
    BlockedBlocked = And(Equals(s['m'], Blocked),
                         Equals(p['m'], Blocked),
                         Equals(p['V'],s['v']),
                         Equals(p['v'], s['t'] + (-Real(aBlocked) * Real(P))),
                         Equals(p['timer'],s['timer']),
                         p['t'] < p['timer'],
                         p['t']>s['t']+delta)
    
    
    
    return Or(StartToFree, FreeToStopping, StoppingToBlocked, BlockedToFree, StoppingToStopped, FreeFree, StoppingStopping, BlockedBlocked)
    

In [5]:
def gera_traco(declare,init,trans,k):

    with Solver(name="z3") as s:
    
        # completar
    
        # cria k copias do estado
        trace = [declare(i) for i in range(k)]
    
        # criar o traço
        s.add_assertion(init(trace[0]))
        for i in range(k-1):
            s.add_assertion(trans(trace[i], trace[i+1]))
        
        if s.solve():
            for i in range(k):
                print()
                print("Modo:",i)
                m = s.get_py_value(trace[i]['m'])
                if m==1:
                    print("Stopping")
                elif m==2:
                    print("Blocked")
                    print("timer =", float(s.get_py_value(trace[i]['timer'])))
                elif m==0:
                    print("Free")
                    print("timer =", float(s.get_py_value(trace[i]['timer'])))
                elif m==3:
                    print("Stopped")
                else:
                    print("Start")
                    
                print("t =", float(s.get_py_value(trace[i]['t'])))
                print("V =", float(s.get_py_value(trace[i]['V'])))
                print("v =", float(s.get_py_value(trace[i]['v'])))
        else:
            print("Solver cannot solve")

In [8]:
gera_traco(declare,init,trans,3)

SolverReturnedUnknownResultError: 

In [7]:
def bmc_always(declare,init,trans,prop,K):
    for k in range(1,K+1):
        with Solver(name="z3") as s:
            
            # completar
            trace = [declare(i) for i in range(k)]
    
            s.add_assertion(init(trace[0]))
            for i in range(k-1):
                s.add_assertion(trans(trace[i], trace[i+1]))
            
            # 
            s.add_assertion(Not(prop(trace[k-1])))
            if s.solve():
                for i in range(k):
                    print()
                    print("State:",i)
                    m = s.get_py_value(trace[i]['m'])
                    if m==1:
                        print("Stopping")
                    elif m==2:
                        print("Blocked")
                        print("timer =", float(s.get_py_value(trace[i]['timer'])))
                    elif m==0:
                        print("Free")
                        print("timer =", float(s.get_py_value(trace[i]['timer'])))
                    elif m==3:
                        print("Stopped")
                    else:
                        print("Start")
                    
                    print("t =", float(s.get_py_value(trace[i]['t'])))
                    print("V =", float(s.get_py_value(trace[i]['V'])))
                    print("v =", float(s.get_py_value(trace[i]['v'])))
                return
                
    print("A propriedade é válida para traços de tamanho até " + str(k))